## Търсене и извличане на информация. Приложение на дълбоко машинно обучение
> ### Стоян Михов
> #### Зимен семестър 2021/2022

### Упражнение 13

 За да работи програмата трябва корпуса от публицистични текстове за Югоизточна Европа,
 да се намира разархивиран в директорията, в която е програмата (виж упражнение 2).

 Преди да се стартира програмата е необходимо да се активира съответното обкръжение с командата: `conda activate tii`

In [1]:
import sys
import nltk
from nltk.corpus import PlaintextCorpusReader
import numpy as np
import torch
import random
import math

######  Визуализация на прогреса

In [2]:
class progressBar:
    def __init__(self ,barWidth = 50):
        self.barWidth = barWidth
        self.period = None
    def start(self, count):
        self.item=0
        self.period = int(count / self.barWidth)
        sys.stdout.write("["+(" " * self.barWidth)+"]")
        sys.stdout.flush()
        sys.stdout.write("\b" * (self.barWidth+1))
    def tick(self):
        if self.item>0 and self.item % self.period == 0:
            sys.stdout.write("-")
            sys.stdout.flush()
        self.item += 1
    def stop(self):
        sys.stdout.write("]\n")

In [3]:
def extractDictionary(corpus, limit=20000):
    pb = progressBar()
    pb.start(len(corpus))
    dictionary = {}
    for doc in corpus:
        pb.tick()
        for w in doc:
            if w not in dictionary: dictionary[w] = 0
        dictionary[w] += 1
    L = sorted([(w,dictionary[w]) for w in dictionary], key = lambda x: x[1] , reverse=True)
    if limit > len(L): limit = len(L)
    words = [ w for w,_ in L[:limit] ] + [unkToken] + [padToken]
    word2ind = { w:i for i,w in enumerate(words)}
    pb.stop()
    return words, word2ind

In [4]:
def splitSentCorpus(fullSentCorpus, testFraction = 0.1):
    random.seed(42)
    random.shuffle(fullSentCorpus)
    testCount = int(len(fullSentCorpus) * testFraction)
    testSentCorpus = fullSentCorpus[:testCount]
    trainSentCorpus = fullSentCorpus[testCount:]
    return testSentCorpus, trainSentCorpus

######   Зареждане на корпуса

In [5]:
corpus_root = '../../JOURNALISM.BG/C-MassMedia'
myCorpus = PlaintextCorpusReader(corpus_root, '.*\.txt')
startToken = '<s>'
endToken = '</s>'
unkToken = '<unk>'
padToken = '<pad>'

In [6]:
corpus = [ [startToken] + [w.lower() for w in sent] + [endToken] for sent in myCorpus.sents()]
words, word2ind = extractDictionary(corpus)

[                                                  --------------------------------------------------]


In [7]:
testCorpus, trainCorpus  = splitSentCorpus(corpus, testFraction = 0.01)

In [8]:
batchSize = 32
emb_size = 50
hid_size = 100

In [9]:
!nvidia-smi

Tue Jan 18 23:10:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.23       Driver Version: 511.23       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   42C    P0    N/A /  75W |    503MiB /  4096MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
torch.cuda.is_available()

True

In [11]:
#device = torch.device("cpu")
device = torch.device("cuda:0")
#device = torch.device("cuda:1")

##### LSTM с пакетиране на партида

In [12]:
class LSTMLanguageModelPack(torch.nn.Module):
    def __init__(self, embed_size, hidden_size, word2ind, unkToken, padToken):
        super(LSTMLanguageModelPack, self).__init__()
        self.word2ind = word2ind
        self.unkTokenIdx = word2ind[unkToken]
        self.padTokenIdx = word2ind[padToken]
        self.lstm = torch.nn.LSTM(embed_size, hidden_size)
        self.embed = torch.nn.Embedding(len(word2ind), embed_size)
        self.projection = torch.nn.Linear(hidden_size,len(word2ind))
    
    def preparePaddedBatch(self, source):
        device = next(self.parameters()).device
        m = max(len(s) for s in source)
        sents = [[self.word2ind.get(w,self.unkTokenIdx) for w in s] for s in source]
        sents_padded = [ s+(m-len(s))*[self.padTokenIdx] for s in sents]
        return torch.t(torch.tensor(sents_padded, dtype=torch.long, device=device)) # (w,s)
    
    def forward(self, source):
        X = self.preparePaddedBatch(source) # (w,s)
        E = self.embed(X[:-1]) # (w,s,e) # cntg
        source_lengths = [len(s)-1 for s in source]
        outputPacked, _ = self.lstm(torch.nn.utils.rnn.pack_padded_sequence(E, source_lengths,enforce_sorted=False))
        output,_ = torch.nn.utils.rnn.pad_packed_sequence(outputPacked) # (w,s,h)

        Z = self.projection(output.flatten(0,1)) # (w*s,h)
        Y_bar = X[1:].flatten(0,1) # (w*s)
        H = torch.nn.functional.cross_entropy(Z,Y_bar,ignore_index=self.padTokenIdx)
        return H

In [13]:
lm = LSTMLanguageModelPack(emb_size, hid_size, word2ind, unkToken, padToken).to(device) # i n
optimizer = torch.optim.Adam(lm.parameters(), lr=0.01)

In [14]:
idx = np.arange(len(trainCorpus), dtype='int32')
np.random.shuffle(idx)

In [15]:
for b in range(0, len(idx), batchSize):
    batch = [ trainCorpus[i] for i in idx[b:min(b+batchSize, len(idx))] ]
    H = lm(batch)
    optimizer.zero_grad()
    H.backward()
    optimizer.step()
    if b % 10 == 0:
        print(b, '/', len(idx), H.item())

0 / 388463 9.89737319946289
160 / 388463 6.993530750274658
320 / 388463 7.019619941711426
480 / 388463 6.388062000274658
640 / 388463 6.440225124359131
800 / 388463 6.282516002655029
960 / 388463 5.968737602233887
1120 / 388463 6.178872585296631
1280 / 388463 6.01486873626709
1440 / 388463 5.940061569213867
1600 / 388463 5.865983963012695
1760 / 388463 5.653263092041016
1920 / 388463 5.64120626449585
2080 / 388463 5.555856704711914
2240 / 388463 5.644376277923584
2400 / 388463 5.311214447021484
2560 / 388463 5.380514621734619
2720 / 388463 5.682637691497803
2880 / 388463 5.534104347229004
3040 / 388463 5.559726715087891
3200 / 388463 5.400074005126953
3360 / 388463 5.478166580200195
3520 / 388463 5.323108673095703
3680 / 388463 5.172706127166748
3840 / 388463 5.278953552246094
4000 / 388463 5.090494632720947
4160 / 388463 5.277441501617432
4320 / 388463 5.009152889251709
4480 / 388463 4.942946910858154
4640 / 388463 4.858317852020264
4800 / 388463 4.846898555755615
4960 / 388463 5.1539

40000 / 388463 3.9563515186309814
40160 / 388463 3.761293888092041
40320 / 388463 3.8995509147644043
40480 / 388463 3.8591508865356445
40640 / 388463 3.8854258060455322
40800 / 388463 3.8320443630218506
40960 / 388463 4.006970405578613
41120 / 388463 3.93361234664917
41280 / 388463 3.8793907165527344
41440 / 388463 4.283437728881836
41600 / 388463 4.03537654876709
41760 / 388463 3.980067253112793
41920 / 388463 3.762220621109009
42080 / 388463 4.037419319152832
42240 / 388463 4.070676803588867
42400 / 388463 4.0283708572387695
42560 / 388463 4.133161544799805
42720 / 388463 3.9122753143310547
42880 / 388463 3.827867269515991
43040 / 388463 4.047313690185547
43200 / 388463 3.9324216842651367
43360 / 388463 3.9516348838806152
43520 / 388463 4.106645584106445
43680 / 388463 3.8975443840026855
43840 / 388463 3.9248857498168945
44000 / 388463 4.173266887664795
44160 / 388463 4.034648895263672
44320 / 388463 4.157316207885742
44480 / 388463 3.89713454246521
44640 / 388463 3.691746711730957
4

79360 / 388463 3.74092435836792
79520 / 388463 3.594423532485962
79680 / 388463 3.679572820663452
79840 / 388463 3.6389169692993164
80000 / 388463 3.9129629135131836
80160 / 388463 3.9419045448303223
80320 / 388463 3.7117855548858643
80480 / 388463 3.8782622814178467
80640 / 388463 3.7391903400421143
80800 / 388463 3.753458261489868
80960 / 388463 3.964487075805664
81120 / 388463 3.7300310134887695
81280 / 388463 3.3936073780059814
81440 / 388463 3.879814624786377
81600 / 388463 3.8664603233337402
81760 / 388463 3.707324981689453
81920 / 388463 3.850999116897583
82080 / 388463 3.7165029048919678
82240 / 388463 3.6656644344329834
82400 / 388463 3.499633312225342
82560 / 388463 3.9974405765533447
82720 / 388463 3.554377555847168
82880 / 388463 3.9032604694366455
83040 / 388463 4.040353298187256
83200 / 388463 3.9348411560058594
83360 / 388463 3.948547601699829
83520 / 388463 3.8359899520874023
83680 / 388463 3.7837493419647217
83840 / 388463 3.6949737071990967
84000 / 388463 3.7253999710

118080 / 388463 3.7235054969787598
118240 / 388463 3.7670738697052
118400 / 388463 3.6279985904693604
118560 / 388463 3.7677102088928223
118720 / 388463 3.623511552810669
118880 / 388463 3.8234570026397705
119040 / 388463 3.5134360790252686
119200 / 388463 3.9765677452087402
119360 / 388463 3.8477282524108887
119520 / 388463 3.801976203918457
119680 / 388463 3.465766191482544
119840 / 388463 3.7235300540924072
120000 / 388463 3.9122588634490967
120160 / 388463 3.6804916858673096
120320 / 388463 3.3895761966705322
120480 / 388463 3.529493570327759
120640 / 388463 3.7490029335021973
120800 / 388463 3.308763027191162
120960 / 388463 3.702219009399414
121120 / 388463 3.782839775085449
121280 / 388463 3.4591546058654785
121440 / 388463 3.393665075302124
121600 / 388463 3.8922150135040283
121760 / 388463 3.7022323608398438
121920 / 388463 3.5143723487854004
122080 / 388463 3.9062671661376953
122240 / 388463 3.6388347148895264
122400 / 388463 3.5659732818603516
122560 / 388463 3.6022999286651

156160 / 388463 3.9078996181488037
156320 / 388463 3.575254440307617
156480 / 388463 3.701608657836914
156640 / 388463 4.009967803955078
156800 / 388463 3.5274055004119873
156960 / 388463 3.7892720699310303
157120 / 388463 3.7613067626953125
157280 / 388463 3.803645133972168
157440 / 388463 3.8468735218048096
157600 / 388463 3.744687080383301
157760 / 388463 3.8340342044830322
157920 / 388463 3.77496600151062
158080 / 388463 3.990816116333008
158240 / 388463 3.353952646255493
158400 / 388463 3.6630303859710693
158560 / 388463 3.5656909942626953
158720 / 388463 3.670877456665039
158880 / 388463 3.753894805908203
159040 / 388463 3.6330127716064453
159200 / 388463 3.734480142593384
159360 / 388463 3.8219985961914062
159520 / 388463 3.5899386405944824
159680 / 388463 3.6933023929595947
159840 / 388463 3.6431260108947754
160000 / 388463 3.4203460216522217
160160 / 388463 3.7840206623077393
160320 / 388463 3.8010413646698
160480 / 388463 3.4566822052001953
160640 / 388463 3.74344539642334
16

RuntimeError: CUDA out of memory. Tried to allocate 942.00 MiB (GPU 0; 4.00 GiB total capacity; 1.89 GiB already allocated; 643.61 MiB free; 2.04 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [15]:
def perplexity(lm, testCorpus, batchSize):
    H = 0.
    c = 0
    for b in range(0,len(testCorpus),batchSize):
        batch = testCorpus[b:min(b+batchSize, len(testCorpus))]
        l = sum(len(s)-1 for s in batch)
        c += l
        with torch.no_grad():
            H += l * lm(batch)
    return math.exp(H/c)

In [16]:
perplexity(lm, testCorpus, batchSize)

33.608086011322

######  Двупосочен LSTM с пакетиране на партида

In [17]:
class BiLSTMLanguageModelPack(torch.nn.Module):
    def __init__(self, embed_size, hidden_size, word2ind, unkToken, padToken, endToken):
        super(BiLSTMLanguageModelPack, self).__init__()
        self.word2ind = word2ind
        self.unkTokenIdx = word2ind[unkToken]
        self.padTokenIdx = word2ind[padToken]
        self.endTokenIdx = word2ind[endToken]
        self.hidden_size = hidden_size
        self.lstm = torch.nn.LSTM(embed_size, hidden_size, bidirectional=True)
        self.embed = torch.nn.Embedding(len(word2ind), embed_size)
        self.projection = torch.nn.Linear(2*hidden_size,len(word2ind))

    def preparePaddedBatch(self, source): #
        device = next(self.parameters()).device
        m = max(len(s) for s in source)
        sents = [[self.word2ind.get(w,self.unkTokenIdx) for w in s] for s in source]
        sents_padded = [ s+(m-len(s))*[self.padTokenIdx] for s in sents]
        return torch.t(torch.tensor(sents_padded, dtype=torch.long, device=device))

    def forward(self, source):
        batch_size = len(source)
        X = self.preparePaddedBatch(source) # (w,s)
        E = self.embed(X) # (w,s,e)
        
        source_lengths = [len(s) for s in source]
        m = X.shape[0]
        outputPacked, _ = self.lstm(torch.nn.utils.rnn.pack_padded_sequence(E, source_lengths,enforce_sorted=False)) #bd
        
        output,_ = torch.nn.utils.rnn.pad_packed_sequence(outputPacked) # (w,s,2h) # d
        output = output.view(m, batch_size, 2, self.hidden_size) # (w,s,2,h) # d
        t = torch.cat((output[:-2,:,0,:], output[2:,:,1,:]),2) # (w,s,2h) # d
        Z = self.projection(t.flatten(0,1)) # (w*s,2h)

        Y_bar = X[1:-1].flatten(0,1)
        Y_bar[Y_bar==self.endTokenIdx] = self.padTokenIdx
        H = torch.nn.functional.cross_entropy(Z,Y_bar,ignore_index=self.padTokenIdx)
        return H

In [18]:
blm = BiLSTMLanguageModelPack(emb_size, hid_size, word2ind, unkToken, padToken, endToken).to(device)
optimizer = torch.optim.Adam(blm.parameters(), lr=0.01)

In [19]:
idx = np.arange(len(trainCorpus), dtype='int32')
np.random.shuffle(idx)

In [ ]:
for b in range(0, len(idx), batchSize):
    batch = [ trainCorpus[i] for i in idx[b:min(b+batchSize, len(idx))] ]
    H = blm(batch)
    optimizer.zero_grad()
    H.backward()
    optimizer.step()
    if b % 10 == 0:
        print(b, '/', len(idx), H.item())

In [21]:
def perplexity(blm, testCorpus, batchSize):
    H = 0.
    c = 0
    for b in range(0,len(testCorpus),batchSize):
        batch = testCorpus[b:min(b+batchSize, len(testCorpus))]
        l = sum(len(s)-2 for s in batch)
        c += l
        with torch.no_grad():
            H += l * blm(batch)
    return math.exp(H/c)

In [22]:
perplexity(blm, testCorpus, batchSize)

11.285752269193237

###### LSTM класификатор на документи

In [23]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, langModel, classesCount):
        super(LSTMClassifier, self).__init__()
        self.langModel = langModel
        self.classProjection = torch.nn.Linear(langModel.lstm.hidden_size,classesCount)
    
    def forward(self, source):
        X = self.langModel.preparePaddedBatch(source)
        E = self.langModel.embed(X[:-1])
        source_lengths = [len(s)-1 for s in source]
        _, (h,_) = self.langModel.lstm(torch.nn.utils.rnn.pack_padded_sequence(E, source_lengths,enforce_sorted=False))
        
        Z = self.classProjection(torch.squeeze(h,dim=0)) # (s,h) -> (s,c)
        return Z

In [24]:
fileNames = myCorpus.fileids()

In [25]:
ecoCorpus = [ [startToken] + [w.lower() for w in myCorpus.words(f)] + [endToken] for f in fileNames if f.find('E-Economy'+'/')==0 ]
milCorpus = [ [startToken] + [w.lower() for w in myCorpus.words(f)] + [endToken] for f in fileNames if f.find('S-Military'+'/')==0 ]
polCorpus = [ [startToken] + [w.lower() for w in myCorpus.words(f)] + [endToken] for f in fileNames if f.find('J-Politics'+'/')==0 ]
culCorpus = [ [startToken] + [w.lower() for w in myCorpus.words(f)] + [endToken] for f in fileNames if f.find('C-Culture'+'/')==0 ]

In [26]:
testEcoCorpus, trainEcoCorpus = splitSentCorpus(ecoCorpus)
testMilCorpus, trainMilCorpus = splitSentCorpus(milCorpus)
testPolCorpus, trainPolCorpus = splitSentCorpus(polCorpus)
testCulCorpus, trainCulCorpus = splitSentCorpus(culCorpus)

In [27]:
trainClassCorpus = trainEcoCorpus + trainMilCorpus + trainPolCorpus + trainCulCorpus

In [28]:
trainY = np.concatenate((
                         np.ones(len(trainEcoCorpus),dtype='int32')*0,
                         np.ones(len(trainMilCorpus),dtype='int32')*1,
                         np.ones(len(trainPolCorpus),dtype='int32')*2,
                         np.ones(len(trainCulCorpus),dtype='int32')*3
                         ))

In [29]:
testY = np.concatenate((
                        np.ones(len(testEcoCorpus),dtype='int32')*0,
                        np.ones(len(testMilCorpus),dtype='int32')*1,
                        np.ones(len(testPolCorpus),dtype='int32')*2,
                        np.ones(len(testCulCorpus),dtype='int32')*3
                        ))

In [30]:
idx = np.arange(len(trainClassCorpus), dtype='int32')

In [31]:
classModel = LSTMClassifier(lm,4).to(device)
optimizer = torch.optim.Adam(classModel.parameters(), lr=0.01)

In [ ]:
np.random.shuffle(idx)
for b in range(0, len(idx), batchSize):
    batch = [ trainClassCorpus[i] for i in idx[b:min(b+batchSize, len(idx))] ]
    target = torch.tensor(trainY[idx[b:min(b+batchSize, len(idx))]], dtype = torch.long, device = device)

    Z = classModel(batch)
    H = torch.nn.functional.cross_entropy(Z,target)

    optimizer.zero_grad()
    H.backward()
    optimizer.step()
    if b % 10 == 0:
        print(b, '/', len(idx), H.item())

In [33]:
testClassCorpus = [ testEcoCorpus, testMilCorpus, testPolCorpus, testCulCorpus ]

In [34]:
def gamma(s):
    with torch.no_grad():
        Z = classModel([s])
        return torch.argmax(Z[0]).item()

In [35]:
def testClassifier(testClassCorpus, gamma):
    L = [ len(c) for c in testClassCorpus ]
    pb = progressBar(50)
    pb.start(sum(L))
    classesCount = len(testClassCorpus)
    confusionMatrix = [ [0] * classesCount for _ in range(classesCount) ]
    for c in range(classesCount):
        for text in testClassCorpus[c]:
            pb.tick()
            c_MAP = gamma(text)
            confusionMatrix[c][c_MAP] += 1
    pb.stop()
    precision = []
    recall = []
    Fscore = []
    for c in range(classesCount):
        extracted = sum(confusionMatrix[x][c] for x in range(classesCount))
        if confusionMatrix[c][c] == 0:
            precision.append(0.0)
            recall.append(0.0)
            Fscore.append(0.0)
        else:
            precision.append( confusionMatrix[c][c] / extracted )
            recall.append( confusionMatrix[c][c] / L[c] )
            Fscore.append((2.0 * precision[c] * recall[c]) / (precision[c] + recall[c]))
    P = sum( L[c] * precision[c] / sum(L) for c in range(classesCount) )
    R = sum( L[c] * recall[c] / sum(L) for c in range(classesCount) )
    F1 = (2*P*R) / (P + R)
    print('=================================================================')
    print('Матрица на обърквания: ')
    for row in confusionMatrix:
        for val in row:
            print('{:4}'.format(val), end = '')
        print()
    print('Прецизност: '+str(precision))
    print('Обхват: '+str(recall))
    print('F-оценка: '+str(Fscore))
    print('Обща презизност: '+str(P))
    print('Общ обхват: '+str(R))
    print('Обща F-оценка: '+str(F1))
    print('=================================================================')
    print()

In [36]:
testClassifier(testClassCorpus, gamma)

[                                                  --------------------------------------------------]
Матрица на обърквания: 
   1  18  46   0
   1 144  11   2
   0  29 705   1
   0   1   2  42
Прецизност: [0.5, 0.75, 0.9227748691099477, 0.9333333333333333]
Обхват: [0.015384615384615385, 0.9113924050632911, 0.9591836734693877, 0.9333333333333333]
F-оценка: [0.029850746268656723, 0.8228571428571428, 0.9406270847231488, 0.9333333333333333]
Обща презизност: 0.8686336279120754
Общ обхват: 0.8893320039880359
Обща F-оценка: 0.8788609640877645



###### Двупосочен LSTM класификатор на документи

In [37]:
class BiLSTMClassifier(torch.nn.Module):
    def __init__(self, langModel, classesCount):
        super(BiLSTMClassifier, self).__init__()
        self.langModel = langModel
        self.classProjection = torch.nn.Linear(2*langModel.hidden_size,classesCount)
    
    def forward(self, source):
        batch_size = len(source)
        X = self.langModel.preparePaddedBatch(source)
        E = self.langModel.embed(X)
        source_lengths = [len(s) for s in source]
        _, (h,c) = self.langModel.lstm(torch.nn.utils.rnn.pack_padded_sequence(E, source_lengths,enforce_sorted=False))
        h = h.view(2,batch_size,self.langModel.hidden_size)
        
        Z = self.classProjection(torch.cat([h[0],h[1]],1)) # (batch_size,2*self.langModel.hidden_size) -> (batch_size,4=|C|)
        return Z

In [38]:
classModelB = BiLSTMClassifier(blm,4).to(device)
optimizer = torch.optim.Adam(classModelB.parameters(), lr=0.01)

In [ ]:
idx = np.arange(len(trainClassCorpus), dtype='int32')
np.random.shuffle(idx)
for b in range(0, len(idx), batchSize):
    batch = [ trainClassCorpus[i] for i in idx[b:min(b+batchSize, len(idx))] ]
    target = torch.tensor(trainY[idx[b:min(b+batchSize, len(idx))]], dtype = torch.long, device = device)
    
    Z = classModelB(batch)
    H = torch.nn.functional.cross_entropy(Z,target)
    
    optimizer.zero_grad()
    H.backward()
    optimizer.step()
    if b % 10 == 0:
        print(b, '/', len(idx), H.item())

In [40]:
def gamma(s):
    with torch.no_grad():
        Z = classModelB([s])
        return torch.argmax(Z[0]).item()

In [41]:
testClassifier(testClassCorpus, gamma)

[                                                  --------------------------------------------------]
Матрица на обърквания: 
  40   3  21   1
  15 135   8   0
   5   5 725   0
   5   0   0  40
Прецизност: [0.6153846153846154, 0.9440559440559441, 0.9615384615384616, 0.975609756097561]
Обхват: [0.6153846153846154, 0.8544303797468354, 0.9863945578231292, 0.8888888888888888]
F-оценка: [0.6153846153846154, 0.8970099667774087, 0.9738079247817326, 0.9302325581395349]
Обща презизност: 0.9369830981216337
Общ обхват: 0.9371884346959122
Обща F-оценка: 0.9370857551603071



###### Конволюционен класификатор на документи

In [42]:
class ConvolutionClassifier(torch.nn.Module):
    def __init__(self, embed, filterSize, filterCount, classesCount, word2ind, unkToken, padToken):
        super(ConvolutionClassifier, self).__init__()
        self.embed = embed
        self.word2ind = word2ind
        self.unkTokenIdx = word2ind[unkToken]
        self.padTokenIdx = word2ind[padToken]
        self.convolution = torch.nn.Conv1d(in_channels=embed.embedding_dim, out_channels=filterCount, kernel_size=filterSize)
        self.dropout = torch.nn.Dropout(0.5)
        self.classProjection = torch.nn.Linear(filterCount,classesCount)
    
    def preparePaddedBatch(self, source):
        device = next(self.parameters()).device
        m = max(len(s) for s in source)
        sents = [[self.word2ind.get(w,self.unkTokenIdx) for w in s] for s in source]
        sents_padded = [ s+(m-len(s))*[self.padTokenIdx] for s in sents]
        return torch.tensor(sents_padded, dtype=torch.long, device=device) # (batch_size,max_sent_len,embed_size)
    
    def forward(self, source):
        X = self.preparePaddedBatch(source)
        
        E = torch.transpose(self.embed(X),1,2) # (s,e,w)
        ### Очаква се Е да е тензор с размер (batch_size, embed_size, max_sent_len)

        U,_ = torch.max(torch.relu(self.convolution(E)), dim=2) # (s,oc,w) # d -> (s,oc)
        Z = self.classProjection(self.dropout(U))
        return Z

In [43]:
EMB = lm.embed

In [44]:
classModelE = ConvolutionClassifier(EMB, 7, 400, 4, word2ind, unkToken, padToken).to(device)
optimizer = torch.optim.Adam(classModelE.parameters(), lr=0.01, weight_decay=0.0002)

In [ ]:
idx = np.arange(len(trainClassCorpus), dtype='int32')
classModelE.train() # d
for epoch in range(10):
    np.random.shuffle(idx)
    for b in range(0, len(idx), batchSize):
        batch = [ trainClassCorpus[i] for i in idx[b:min(b+batchSize, len(idx))] ]
        target = torch.tensor(trainY[idx[b:min(b+batchSize, len(idx))]], dtype = torch.long, device = device)
    
        Z = classModelE(batch)
        H = torch.nn.functional.cross_entropy(Z,target)
    
        optimizer.zero_grad()
        H.backward()
        optimizer.step()
        if b % 10 == 0:
            print(b, '/', len(idx), H.item())
classModelE.eval()
testClassifier(testClassCorpus, gamma)

In [46]:
def gamma(s):
    with torch.no_grad():
        Z = classModelE([s])
        return torch.argmax(Z[0]).item()

In [47]:
testClassifier(testClassCorpus, gamma)

[                                                  --------------------------------------------------]
Матрица на обърквания: 
  45   0  20   0
   0 146  12   0
   0   3 732   0
   0   0   2  43
Прецизност: [1.0, 0.9798657718120806, 0.9556135770234987, 1.0]
Обхват: [0.6923076923076923, 0.9240506329113924, 0.9959183673469387, 0.9555555555555556]
F-оценка: [0.8181818181818181, 0.9511400651465798, 0.9753497668221185, 0.9772727272727273]
Обща презизност: 0.9643018654621938
Общ обхват: 0.9631106679960121
Обща F-оценка: 0.9637058986316202



###### LSTM с посимволово влагане с КНН и пакетиране на партида

In [48]:
class CharEmbedding(torch.nn.Module):
    def __init__(self, word2ind, char_embed_size, word_embed_size, filter_size=5, dropoutrate=0.3, padding=1):
        super(CharEmbedding, self).__init__()
        self.word2ind = word2ind
        self.char_embed_size = char_embed_size
        self.word_embed_size = word_embed_size
        self.filter_size = filter_size
        self.dropoutrate = dropoutrate
        self.padding = padding

        alphabetSet = {c for w in word2ind for c in w}
        alphabet = ['§','`','~','№']+list(alphabetSet)
        self.char2id = {c:i for i, c in enumerate(alphabet) }
        self.char_pad = self.char2id['§']
        self.start_of_word = self.char2id['`']
        self.end_of_word = self.char2id['~']
        self.char_unk = self.char2id['№']

        self.CharEmbedding = torch.nn.Embedding(len(self.char2id),self.char_embed_size, padding_idx = self.char_pad)
        self.conv = torch.nn.Conv1d(char_embed_size, word_embed_size, filter_size, padding=padding)
        self.highway_proj = torch.nn.Linear(word_embed_size,word_embed_size)
        self.highway_gate = torch.nn.Linear(word_embed_size,word_embed_size)

        self.Dropout = torch.nn.Dropout(dropoutrate)

    def preparePaddedBatch(self, source):
        device = next(self.parameters()).device
        source_ids = [[ [self.start_of_word] + [self.char2id.get(c, self.char_unk) for c in w ] + [self.end_of_word] for w in s] for s in source]

        max_word_length = max(len(w) for s in source_ids for w in s )
        max_sent_len = max(len(s) for s in source_ids)
    
        sents_padded = []
        for sentence in source_ids:
            sent_padded = [ w + [self.char_pad]*(max_word_length-len(w)) for w in sentence ] + [[self.char_pad]*max_word_length] * (max_sent_len - len(sentence))
            sents_padded.append(sent_padded)

        return torch.transpose(torch.tensor(sents_padded, dtype=torch.long, device=device),0,1).contiguous()

    def forward(self, source):
        batch_size = len(source) # (s,w,c)
        X = self.preparePaddedBatch(source) # (w,s,c)
        X_emb = self.CharEmbedding(X).transpose(2,3) # (w,s,ce,c)

        x_conv = self.conv(X_emb.flatten(0,1)) # (w*s,we,c)
        x_conv_out0,_ = torch.max(torch.nn.functional.relu(x_conv),dim=2) # (w*s,we)
        x_conv_out = x_conv_out0.view((-1,batch_size,self.word_embed_size)) # (w,s,we)

        x_proj = torch.nn.functional.relu(self.highway_proj(x_conv_out)) # (w,s,we)
        x_gate = torch.sigmoid(self.highway_gate(x_conv_out)) # (w,s,we)
        x_highway = x_gate * x_proj + (1 - x_gate) * x_conv_out # (w,s,we)

        output = self.Dropout(x_highway)
        return output

In [49]:
class CharCNNLSTMLanguageModelPack(torch.nn.Module):
    def __init__(self, word_embed_size, hidden_size, word2ind, unkToken, padToken, char_embed_size, filter_size=5, dropoutrate=0.3, padding=1):
        super(CharCNNLSTMLanguageModelPack, self).__init__()
        self.word2ind = word2ind
        self.unkTokenIdx = word2ind[unkToken]
        self.padTokenIdx = word2ind[padToken]

        self.charEmbedding = CharEmbedding(word2ind, char_embed_size, word_embed_size, filter_size, dropoutrate, padding)
        self.lstm = torch.nn.LSTM(word_embed_size, hidden_size)
        self.projection = torch.nn.Linear(hidden_size,len(word2ind))
    
    def preparePaddedBatch(self, source):
        device = next(self.parameters()).device
        m = max(len(s) for s in source)
        sents = [[self.word2ind.get(w,self.unkTokenIdx) for w in s] for s in source]
        sents_padded = [ s+(m-len(s))*[self.padTokenIdx] for s in sents]
        return torch.t(torch.tensor(sents_padded, dtype=torch.long, device=device))
    
    def forward(self, source):
        X = self.preparePaddedBatch(source)
        E = self.charEmbedding(source)
        source_lengths = [len(s)-1 for s in source]
        outputPacked, _ = self.lstm(torch.nn.utils.rnn.pack_padded_sequence(E, source_lengths,enforce_sorted=False))
        output,_ = torch.nn.utils.rnn.pad_packed_sequence(outputPacked)
        
        Z = self.projection(output.flatten(0,1))
        Y_bar = X[1:].flatten(0,1)
        H = torch.nn.functional.cross_entropy(Z,Y_bar,ignore_index=self.padTokenIdx)
        return H

In [50]:
clm = CharCNNLSTMLanguageModelPack(256, 256, word2ind, unkToken, padToken, 32).to(device)
optimizer = torch.optim.Adam(clm.parameters(), lr=0.001)

In [51]:
idx = np.arange(len(trainCorpus), dtype='int32')
np.random.shuffle(idx)

In [ ]:
trainCorpus

In [ ]:
clm.train()
for b in range(0, len(idx), batchSize):
    batch = [ trainCorpus[i] for i in idx[b:min(b+batchSize, len(idx))] ]
    H = clm(batch)
    optimizer.zero_grad()
    H.backward()
    optimizer.step()
    if b % 10 == 0:
        print(b, '/', len(idx), H.item())
clm.eval()
perplexity(clm, testCorpus, batchSize)

######  Двупосочен LSTM с пакетиране на партида

In [54]:
class CharCNNBiLSTMLanguageModelPack(torch.nn.Module):
    def __init__(self, word_embed_size, hidden_size, word2ind, unkToken, padToken, endToken, char_embed_size, filter_size=5, dropoutrate=0.3, padding=1):
        super(CharCNNBiLSTMLanguageModelPack, self).__init__()
        self.word2ind = word2ind
        self.unkTokenIdx = word2ind[unkToken]
        self.padTokenIdx = word2ind[padToken]
        self.endTokenIdx = word2ind[endToken]
        self.hidden_size = hidden_size

        self.charEmbedding = CharEmbedding(word2ind, char_embed_size, word_embed_size, filter_size, dropoutrate, padding)
        self.lstm = torch.nn.LSTM(word_embed_size, hidden_size, bidirectional=True)
        self.projection = torch.nn.Linear(2*hidden_size,len(word2ind))
    
    def preparePaddedBatch(self, source):
        device = next(self.parameters()).device
        m = max(len(s) for s in source)
        sents = [[self.word2ind.get(w,self.unkTokenIdx) for w in s] for s in source]
        sents_padded = [ s+(m-len(s))*[self.padTokenIdx] for s in sents]
        return torch.t(torch.tensor(sents_padded, dtype=torch.long, device=device))
    
    def forward(self, source):
        batch_size = len(source)
        X = self.preparePaddedBatch(source)
        E = self.charEmbedding(source)

        source_lengths = [len(s) for s in source]
        m = X.shape[0]
        outputPacked, _ = self.lstm(torch.nn.utils.rnn.pack_padded_sequence(E, source_lengths,enforce_sorted=False))
        
        output,_ = torch.nn.utils.rnn.pad_packed_sequence(outputPacked)
        output = output.view(m, batch_size, 2, self.hidden_size)
        t = torch.cat((output[:-2,:,0,:], output[2:,:,1,:]),2)
        Z = self.projection(t.flatten(0,1))
        
        Y_bar = X[1:-1].flatten(0,1)
        Y_bar[Y_bar==self.endTokenIdx] = self.padTokenIdx
        H = torch.nn.functional.cross_entropy(Z,Y_bar,ignore_index=self.padTokenIdx)
        return H

In [55]:
cblm = CharCNNBiLSTMLanguageModelPack(256, 256, word2ind, unkToken, padToken, endToken, 32).to(device)
optimizer = torch.optim.Adam(cblm.parameters(), lr=0.001)

In [56]:
idx = np.arange(len(trainCorpus), dtype='int32')
np.random.shuffle(idx)

In [ ]:
cblm.train()
for b in range(0, len(idx), batchSize):
    batch = [ trainCorpus[i] for i in idx[b:min(b+batchSize, len(idx))] ]
    H = cblm(batch)
    optimizer.zero_grad()
    H.backward()
    optimizer.step()
    if b % 10 == 0:
        print(b, '/', len(idx), H.item())

In [58]:
perplexity(cblm, testCorpus, batchSize)

10.162559473913722